# Part 6:  Codes secrets et échange de clés en Python 3.x

Dans ce notebook, nous introduisons la cryptographie -- comment communiquer en toute sécurité sur des canaux non sécurisés. Nous commençons par l’étude de deux méthodes simples de code secret (appelé aussi chiffrement), le code César et sa variante la plus sophistiquée, le code Vigenère. Le code Vigenère utilise une **clé** pour transformer le **texte en clair** (c’est-à-dire le message) en **texte chiffré** (le message codé) et utilise la même clé pour transformer le texte chiffré en texte en clair. Par conséquent, deux parties peuvent communiquer en toute sécurité si elles - et elles seules - possèdent la clé.

Si la sécurité de la communication repose sur la possession d'une clé commune, il nous reste un nouveau problème: comment les deux parties s'accordent-elles sur une clé commune, surtout si elles sont éloignées l'une de l'autre et communiquent par un canal peu sécurisé?

Une solution intelligente à ce problème a été publiée en 1976 par Whitfield Diffie et Martin Hellman, et s'appelle donc échange de clés Diffie-Hellman. Elle tire parti de l'arithmétique modulaire et repose sur l'existence d'une racine primitive (modulo un nombre premier) et sur la difficulté à résoudre le **problème du logarithme discret**.

## Table des matières

- [Code secret](#cipher)
- [Echange de clés](#keyexchange)


<a id='cipher'></a>

## Codes

Un chiffrement est un moyen de transformer un message, appelé **texte en clair**, en une forme différente, le **texte chiffré**, qui masque le sens à tout le monde, sauf au destinataire. Par exemple, dans un **code par substitution** on change chaque lettre de l'alphabet en une autre lettre. Ce n'est pas trop difficile à résoudre à la main et c'est facile à résoudre pour les ordinateurs, grâce à l’analyse de fréquence (compréhension de la fréquence à laquelle différentes lettres ou combinaisons de lettres se produisent).


### Caractères ASCII et le code César

Même si les codes par substitution sont faciles à briser, ils constituent un bon point de départ. Pour implémenter des chiffrements par substitution en Python, nous devons étudier le type *string* (chaine de caractères) plus en détail. Pour déclarer une chaîne de caractères, il suffit de mettre la chaîne entre guillemets. Vous pouvez utiliser des lettres, des chiffres, des espaces et de nombreux symboles dans une chaîne. Vous pouvez entourer votre chaîne de caractères par des guillemets simples, comme `'Hello'` ou des guillemets doubles, comme `"Bonjour"`. Cette flexibilité est pratique si vous souhaitez utiliser des guillemets dans la chaîne. Par exemple, la chaîne *C'est un nombre premier* devrait être décrite en Python avec des guillemets `"C'est un nombre premier"` afin que l'apostrophe ne se confonde pas avec les guillemets.

Les chaînes sont indexées comme des listes de caractères.

In [ ]:
W = "Hello"
print(W)
for j in range(len(W)):  # len(W) is the length of the string W.
    print(W[j])  # Access the jth character of the string.

 Un caractère est encore de type string ; c'est une chaîne de longueur 1.

In [ ]:
print(type(W))
print(type(W[0]))  # W[0] is a character.

Comme les ordinateurs stockent les données en binaire, les concepteurs des premiers ordinateurs (dans les années 1960) ont créé un code appelé [**ASCII**](https://en.wikipedia.org/wiki/ASCII) (American Standard Code for Information Interchange) qui associe à chaque caractère un nombre compris entre 0 et 127. Chaque nombre compris entre 0 et 127 est représenté en binaire par 7 bits (entre 0000000 et 1111111), ainsi chaque caractère est stocké avec 7 bits de mémoire. Plus tard, l'ASCII a été étendu avec 128 caractères supplémentaires, de sorte que les codes compris entre 0 et 255 ont été utilisés, nécessitant 8 bits. Une suite de 8 bits de mémoire est appelée **octet**. Un octet de mémoire suffit pour stocker un caractère ASCII étendu.

Vous remarquerez peut-être que 256 caractères ASCII sont disponibles, mais votre clavier compte moins de 256 caractères, même une fois que vous avez inclus des symboles tels que `#` et `;`. Certains de ces caractères supplémentaires sont destinés aux lettres accentuées, tandis que d'autres sont des reliques d'anciens ordinateurs. Si vous êtes curieux, vous pouvez rechercher un [tableau ASCII complet](http://www.asciitable.com/).

De nos jours, la communauté mondiale des utilisateurs nécessite bien plus que 256 caractères - il existe de nombreux alphabets dans le monde! Ainsi, au lieu d’ASCII, nous pouvons accéder à plus de 100 000 **caractères Unicode**. Faites défiler une [table unicode](https://unicode-table.com/en/) pour voir ce qui est possible. Avec emoji, les tables unicode sont entrées dans un [territoire inexploré](https://xkcd.com/1813/). Python version 3.x prend totalement en charge le format Unicode.

Mais ici, nous restons dans le code ASCII à l’ancienne, car il suffit pour les messages de base en anglais. Python a les commandes intégrées `chr` et` ord` pour la conversion numéro (0--255) en caractère et inversement.

In [ ]:
chr(65)

In [ ]:
ord('A')

Le code suivant produira une table des caractères ASCII avec des numéros compris entre 32 et 126. Il s’agit d’une plage qui inclut tous les caractères et symboles anglais les plus courants sur un clavier US. Notez que le numéro ASCII 32 correspond à un espace vide (un caractère important pour les longs messages!)

In [ ]:
for a in range(32,127):
    c = chr(a)
    print("ASCII {} is {}".format(a, c))

Puisque nous travaillerons uniquement avec la plage ASCII comprise entre 32 et 126, il sera utile de "passer" d'autres nombres dans cette plage. Par exemple, nous interpréterons 127 comme 32, 128 comme 33, etc., lorsque nous convertirons des nombres en dehors de la plage considérée.

La fonction suivante force un nombre dans une plage donnée, en utilisant l'opérateur mod. C'est une astuce courante: faire en sorte que les listes tournent cycliquement.

In [ ]:
def inrange(n,range_min, range_max):
    '''
    The input number n can be any integer.
    The output number will be between range_min and range_max (inclusive)
    If the input number is already within range, it will not change.
    '''
    range_len = range_max - range_min + 1
    a = n % range_len
    if a < range_min:
        a = a + range_len
    return a

In [ ]:
inrange(13, 1, 10)

In [ ]:
inrange(17, 5, 50)

Nous pouvons maintenant mettre en place un **chiffrement par substitution** en convertissant les caractères en leurs numéros ASCII, en mélangeant les numéros et en reconvertissant. L'un des plus simples algorithmes de substitution s'appelle **code César**, dans lequel chaque caractère est décalé d'un montant fixe. Par exemple, un code César avec décalage 3 enverrait «A» à «D» et «B» à «E», etc. À la fin de la liste les caractères sont décalés cycliquement, en utilisant la fonction `inrange`.

#### Exercice
Ecrire une fonction `Caesar_shift`, qui prend un caractère ASCII `c` (dont le numéro est compris entre 32 et 126) et qui renvoie le caractère décalé, au sens du code César, du montant `shift`.

In [ ]:
def Caesar_shift(c, shift):
    '''
    Shifts the character c by shift units
    within the ASCII table between 32 and 126. 
    The shift parameter can be any integer!
    '''
    # code de la fonction Caesar_shift ici

Voyons l'effet du code de César sur notre table ASCII.

In [ ]:
for a in range(32, 127):
    c = chr(a)
    print("ASCII {} is {}, which shifts to {}".format(a, c, Caesar_shift(c, 5))) # Shift by 5.

#### Exercice
Ecrire une fonction `Caesar_cipher` qui prend en arguments une chaine de caractères `plaintext`, le message en clair, ainsi qu'un entier `shift`, et qui renvoie le message chiffré par un code César de décalage `shift`.

In [ ]:
def Caesar_cipher(plaintext, shift):
    # code de la fonction Caesar_cipher ici

In [ ]:
print(Caesar_cipher('Hello! Can you read this?', 5))  # Shift forward 5 units in ASCII.

Comme prévu, le chiffrement Caesar transforme le texte en clair en texte chiffré, en utilisant un décalage de la table ASCII.
Pour déchiffrer le texte chiffré, on utilise à nouveau le code César, avec le décalage négatif.

In [ ]:
print(Caesar_cipher('Mjqqt&%%Hfs%~tz%wjfi%ymnxD', -5))  # Shift back 5 units in ASCII.

### Le code Vigenère

Le chiffrement de César est assez facile à casser, par une attaque en force brute (décalage de toutes les valeurs possibles) ou une analyse de fréquence (comparez la fréquence des caractères dans un message à la fréquence des caractères dans les messages anglais typiques).
Le chiffrement Vigenère est une variante du chiffrement Caesar qui utilise une **clé de chiffrement** pour faire varier le paramètre de décalage tout au long du processus de chiffrement. Par exemple, pour chiffrer le message "This is very secret" à l'aide de la clé "Key", vous devez aligner les caractères du message au-dessus des copies répétées de la clé.

T | h | i | s |   | i | s |   | v | e | r | y |   | s | e | c | r | e | t
--|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|--
K | e | y | K | e | y | K | e | y | K | e | y | K | e | y | K | e | y | K

Ensuite, vous transformez tout en ASCII et utilisez la rangée inférieure pour décaler la rangée supérieure.

ASCII message | 84 | 104 | 105 | 115 | 32 | 105 | 115 | 32 | 118 | 101 | 114 | 121 | 32 | 115 | 101 | 99 | 114 | 101 | 116 
---|-----|-----
Shift | 75 | 101 | 121 | 75 | 101 | 121 | 75 | 101 | 121 | 75 | 101 | 121 | 75 | 101 | 121 | 75 | 101 | 121 | 75 
ASCII shifted | 159 | 205 | 226 | 190 | 133 | 226 | 190 | 133 | 239 | 176 | 215 | 242 | 107 | 216 | 222 | 174 | 215 | 222 | 191 
ASCII shifted in range |   64 | 110 | 36 | 95 | 38 | 36 | 95 | 38 | 49 | 81 | 120 | 52 | 107 | 121 | 32 | 79 | 120 | 32 | 96 

Enfin, les numéros ASCII décalés sont reconvertis en caractères pour la transmission. Dans ce cas, les numéros 64, 110, 36, 95, etc., sont convertis en texte chiffré ``"@n$_&$_&1Qx4ky Ox \`"``

Le chiffre de Vigenère est beaucoup plus difficile à déchiffrer que le chiffre de César, si vous n'avez pas la clé. En effet, les variations de décalage rendent l'analyse de fréquence plus difficile. 

Le chiffrement de Vigenère est cependant considéré comme faible avec les normes actuelles 
(voir [Wikipedia](https://en.wikipedia.org/wiki/Vigen%C3%A8re_cipher#Cryptanalysis) pour une description des attaques au 19ème siècle), mais il illustre les ingrédients de base d'un cryptosystème avec **clé symétrique** : le texte en clair, le texte chiffré et une clé unique. Aujourd'hui, les cryptosystèmes à clé symétrique tels que 
[AES](https://en.wikipedia.org/wiki/Advanced_Encryption_Standard) et 
[3DES](https://en.wikipedia.org/wiki/Triple_DES) sont utilisés en permanence pour la communication sécuririsée.

#### Exercice
Ecrire une fonction `Vigenere_cipher` qui prend en arguments un message, en clair, et une clé, et qui renvoie le message codé par Vigenère.

In [ ]:
def Vigenere_cipher(plaintext, key):
    # code fonction Vigenere_cipher ici

In [ ]:
print(Vigenere_cipher('This is very secret', 'Key')) # 'Key' is probably a bad key!!

Le chiffrement de Vigenère est appelé cryptosystème **symétrique**, car la même clé que celle utilisée pour chiffrer le texte en clair peut être utilisée pour déchiffrer le texte chiffré.

#### Exercice
Ecrire une fonction `Vigenere_decipher` qui prend en arguments un message codé par Vigenère, ainsi que la clé, et qui renvoie le message décodé.

In [ ]:
def Vigenere_decipher(ciphertext, key):
    # code fonction Vigenere_decipher ici

In [ ]:
print(Vigenere_decipher('@n$_&$_&1Qx4ky Ox `', 'Key'))

In [ ]:
# Try a few cipher/deciphers yourself to get used to the Vigenere system.
# Essayez quelques cipher/deciphers par vous-même pour vous habituer au cryptosystème Vigenere.

Le chiffrement Vigenère devient un moyen efficace pour une communication sécurisée entre deux parties, à condition de partager une clé secrète. Au 19ème siècle, cela signifiait souvent que les parties nécessitaient une première réunion en personne pour se mettre d'accord sur une clé, ou qu'un messager bien gardé la porterait d'une partie à l'autre.

Aujourd'hui, comme nous souhaitons communiquer régulièrement et en toute sécurité sur de longues distances, il est plus difficile de s'entendre sur une clé. Cela ressemble au problème de la poule et de l'œuf, où nous avons besoin d'un secret partagé pour communiquer de manière sécurisée, mais nous ne pouvons pas partager un secret sans communiquer de manière sécurisée en premier lieu!

Il est remarquable que ce problème de partage de secrets peut être résolu avec quelques astuces d'arithmétique modulaire. C'est le sujet de la section suivante.

### Exercises

1. Un chiffrement César a été utilisé pour coder un message, avec le texte chiffré suivant: `'j!\'1r$v1"$v&&+1t}v(v$2'`. 
Utilisez une boucle (attaque par force brute) pour déterminer le message d'origine.

2. Imaginez que vous chiffriez un long message (par exemple 1000 mots d’anglais standard) avec un chiffrement Vigenère. Comment pourriez-vous détecter la *longueur* de la clé, si elle est courte (par exemple 3 ou 4 caractères)?

3. Envisagez de coder un message en texte clair par Vigenère avec une clé à 3 caractères, puis recoder le texte chiffré avec une clé à 4 caractères. Expliquez en quoi cela équivaut à coder le message d'origine via un seul chiffrement avec une clé de 12 caractères.

<a id='keyexchange'></a>

## Echange de clés

Nous étudions maintenant l’échange de clés **Diffie-Hellman**, un moyen remarquable pour deux parties de partager un secret sans jamais avoir besoin de le communiquer directement. Leur méthode est basée sur les propriétés d’exponentiation modulaire et l’existence d’une **racine primitive** modulo un nombre premier.

### Racines primitives et nombres premiers de Sophie Germain

Si $p$ est un nombre premier et $GCD(a, p) = 1$, le petit théorème de Fermat dit que 
$$a^{p-1} \equiv 1 \text{ mod } p$$
Il peut cependant arriver que  $a^\ell \equiv 1$ mod $p$ pour une plus petite valeur (positive) de $\ell$. La plus petite valeur positive de $\ell$ est appelée l'**ordre** (ordre multiplicatif, pour être précis) de $a$ modulo $p$, et il s'agit toujours d'un diviseur de $p-1$.

Le code suivant détermine l'ordre d'un nombre, mod $p$, avec une approche par force brute.

In [ ]:
def mult_order(a, p):
    '''
    Determines the (multiplicative) order of an integer
    a, modulo p.  Here p is prime, and GCD(a,p) = 1.
    If bad inputs are used, this might lead to a 
    never-ending loop!
    '''
    current_number = a % p
    current_exponent = 1
    while current_number != 1:
        current_number = (current_number * a)%p
        current_exponent = current_exponent + 1
    return current_exponent
    

In [ ]:
p = 37
for j in range(2, p):
    print("The multiplicative order of {} modulo {} is {}".format(j, p, mult_order(j, p)))
    # These orders should all be divisors of p-1.

Un théorème de Gauss stipule que, si $p$ est premier, alors il existe un entier $b$ dont l'ordre est précisément $p-1$ (le plus grand possible!). Un tel entier s'appelle une **racine primitive** modulo $p$. Par exemple, le calcul précédent a trouvé 12 racines primitives modulo $37$: 2, 5, 13, 15, 17, 18, 19, 20, 22, 24, 32, 35.
Pour illustrer ça (mod 37), consultez [cette affiche](https://fineartamerica.com/featured/epicycles-modulo-37-martin-weissman.html).

Pour tout ce qui suit, supposons que $p$ est un nombre premier. Non seulement des racines primitives *existent* mod $p$, mais elles sont assez courantes. En fait, le nombre de racines primitives mod $p$ est égal à $\phi(p-1)$, où $\phi$ indique l'[indicatrice d'Euler](https://fr.wikipedia.org/wiki/Indicatrice_d%27Euler). En moyenne, $\phi(n)$ correspond à environ $6 / \pi^2$ fois $n$ (pour les entiers positifs $n$). Bien que les nombres de la forme $p-1$ ne soient pas "moyens", on s’attend quand même à ce que $\phi(p-1)$ ne soit pas une très petite fraction de $p-1$.
Vous ne devriez pas avoir à chercher très loin si vous voulez *trouver* une racine primitive.

La partie la plus difficile, en pratique, consiste à déterminer si un nombre $b$ est ou non une racine primitive modulo $p$. Lorsque $p$ est très grand (comme des centaines ou des milliers de chiffres), $p-1$ est également très grand. Il n’est certainement pas pratique d’utiliser tous les puissances (de $1$ à  $p-1$) de $b$ pour déterminer si $b$ est une racine primitive!
La meilleure approche, parfois, consiste à utiliser le fait que l'ordre multiplicatif de $b$ doit être un **diviseur** de $p-1$. Si on peut trouver tous les diviseurs de $p-1$, il suffit de vérifier si $b^d \equiv 1$ mod $p$ pour chaque diviseur $d$. Cela rend le problème de déterminer si $b$ est une racine primitive à peu près aussi difficile que le problème de la factorisation de $p-1$. C'est un problème difficile, en général!

Mais pour l'application qui nous intéresse ici, nous voulons juste un grand nombre premier $p$ **et** une racine primitive mod $p$. Pour ce faire, le plus simple consiste à utiliser un nombre premier de **Sophie Germain** ; c'est un nombre premier $q$ tel que $2q + 1$ est également premier. Lorsque $q$ est un nombre premier de Sophie Germain, le nombre premier résultant $p = 2q + 1$ est appelé **nombre premier sûr**.

Observez que lorsque $p$ est un nombre premier sûr, la décomposition en facteurs premiers de de $p-1$ est
$p-1 = 2q$.

C'est tout. Ainsi, les ordres multiplicatifs possibles d'un élément $b$, mod $p$, sont les diviseurs de $2q$, qui sont $ 1, 2, q, 2q $
Pour vérifier si $b$ est une racine primitive, modulo un nombre premier sûr, $p = 2q + 1$, nous devons vérifier seulement trois choses: est-ce que $b \equiv 1$, ou $b^2 \equiv 1$, ou $b^q \equiv 1$, mod $p$? Si la réponse à ces trois questions est NON, alors $b$ est une racine primitive mod $p$.

In [ ]:
def is_primroot_safe(b, p):
    '''
    Checks whether b is a primitive root modulo p ; p must be a safe prime !! 
    If p is not safe, the results will not be good!
    '''
    q = (p-1) // 2   # q is the Sophie Germain prime
    if b%p == 1:  # Is the multiplicative order 1?
        return False
    if (b*b)%p == 1:  # Is the multiplicative order 2?
        return False
    if pow(b,q,p) == 1:  # Is the multiplicative order q?
        return False
    return True  # If not, then b is a primitive root mod p.

In [ ]:
q = 11 # Sophie Germain prime
p = 2*q + 1 # safe prime
for b in range(2, p):
    print("{}    {}".format(b, is_primroot_safe(b, p)))

A comparer avec le calcul suivant

In [ ]:
p = 23
for j in range(2, p):
    print("The multiplicative order of {} modulo {} is {}".format(j, p, mult_order(j, p)))
    # These orders should all be divisors of p-1.

Cela ne serait pas très utile si nous ne pouvions pas trouver facilement des nombres premiers de Sophie Germain. Heureusement, ils ne sont pas si rares. Les premiers sont 2, 3, 5, 11, 23, 29, 41, 53, 83, 89. On pense, mais ce n’est pas prouvé, qu’il existe une infinité de nombres premiers de Sophie Germain. En pratique, ils sont assez nombreux. Si nous considérons des nombres de magnitude $N$, environ $ 1 / \log(N) $ sont premiers. Parmi ces nombres premiers, on s'attend à ce que $ 1.3 / \log(N) $ soient Sophie Germain. De cette façon, on peut s’attendre à tomber sur les nombres premiers de Sophie Germain si on cherche un peu (et si $ \log(N)^2 $ n’est pas trop grand).
Le code ci-dessous teste si un nombre $ p $ est un nombre premier de Sophie Germain. Nous le construisons en testant simplement si $ p $ et $ 2p + 1 $ sont tous les deux premiers. Nous utilisons le test Miller-Rabin (voir notebook 5) pour vérifier si les nombres sont premiers.

In [ ]:
from random import randint # randint chooses random integers.

def Miller_Rabin(p, base):
    '''
    Tests whether p is prime, using the given base.
    The result False implies that p is definitely not prime.
    The result True implies that p **might** be prime.
    It is not a perfect test!
    '''
    # rappel de la fonction Miller_Rabin ici

def is_prime(p, witnesses=50):  # witnesses is a parameter with a default value of 50.
    '''
    Tests whether a positive integer p is prime.
    For p < 2^64, the test is deterministic, using known good witnesses.
    Good witnesses come from a table at Wikipedia's article on the Miller-Rabin test,
    based on research by Pomerance, Selfridge and Wagstaff, Jaeschke, Jiang and Deng.
    For larger p, a number (by default, 50) of witnesses are chosen at random.
    '''
    # rappel de la fonction is_prime, basée sur le test de Miller_Rabin

In [ ]:
def is_SGprime(p):
    '''
    Tests whether p is a Sophie Germain prime
    '''
    if is_prime(p):  # A bit faster to check whether p is prime first.
        if is_prime(2*p + 1):  # and *then* check whether 2p+1 is prime.
            return True
    return False

Essayons de trouver les nombres de Sophie Germain jusqu'à 100, et leur nombres premiers sûrs associés.

In [ ]:
for j in range(1, 100):
    if is_SGprime(j):
        print(j, 2*j+1)

Nous retrouvons ensuite le premier nombre de Sophie Germain à 100 chiffres! Cela pourrait prendre une minute!

In [ ]:
test_number = 10**99 # Start looking at the first 100-digit number, which is 10^99.
while not is_SGprime(test_number):
    test_number = test_number + 1
print(test_number)

En quelques secondes ou minutes, votre ordinateur teste la primalité de près de 90000 entiers, chacun comportant 100 chiffres. Pas mal!

### Le protocole Diffie-Hellman

Lorsque nous étudions des protocoles pour une communication sécurisée, nous devons examiner les parties qui communiquent (souvent appelées Alice et Bob), et **qui** a connaissance de **quelle** information. Nous supposons à tout moment que le "fil" entre Alice et Bob est mis sur écoute - tout ce qu'ils se disent est activement surveillé, et est donc rendu **public**. Nous supposons également que ce qui se passe sur l'ordinateur d'Alice est privé pour Alice et que ce qui se passe sur l'ordinateur de Bob est privé pour Bob.
Bien sûr, ces deux dernières hypothèses sont optimistes - il y a le danger des virus informatiques qui infectent les ordinateurs et peuvent violer une telle confidentialité!

L'objectif du protocole Diffie-Hellman est - à la fin du processus - qu'Alice et Bob **partagent un secret** sans jamais l'avoir communiqué l'un à l'autre. Le processus implique une série de calculs arithmétiques modulaires effectués sur chacun des ordinateurs d'Alice et de Bob.

Le processus commence lorsque Alice ou Bob crée et publie un **grand nombre premier** `p` et une **racine primitive** `g` modulo `p`. Il est préférable, pour des raisons d'efficacité et de sécurité, de choisir un nombre premier `p` **sûr**. Alice et Bob peuvent créer leur propre nombre premier sûr, ou en choisir un dans une liste publique en ligne, par exemple à partir du [mémo RFC 3526](https://tools.ietf.org/html/rfc3526). De nos jours, il est courant de prendre `p` avec 2048 bits, c’est-à-dire un nombre premier compris entre $2^{2047}$ et $2^{2048}$ (un nombre avec 617 chiffres décimaux!).

Pour les besoins de cette introduction, nous utilisons un nombre premier sûr plus petit, d'environ 256 bits. Nous utilisons la fonctionnalité `SystemRandom` du package` random` pour créer un bon nombre premier aléatoire. Ce n'est pas tellement un problème ici, mais en général, il faut être très prudent en cryptographie que les nombres "aléatoires" soient vraiment "aléatoires"!
La fonction `SystemRandom` utilise les propriétés chaotiques des entrailles de votre ordinateur pour initialiser un générateur de nombres aléatoires. Elle est considérée comme offrant une bonne sécurité sur le plan cryptographique.

In [ ]:
from random import SystemRandom # Import the necessary package.

r = SystemRandom().getrandbits(256)
print("The random integer is {}".format(r))
print("with binary expansion {}".format(bin(r)))  #  r is an integer constructed from 256 random bits.
print("with bit-length {}.".format(len(bin(r)) - 2))  # In case you want to check.  Remember '0b' is at the beginning.

In [ ]:
def getrandSGprime(bitlength):
    '''
    Creates a random Sophie Germain prime p with about 
    bitlength bits.
    '''
    while True:
        p = SystemRandom().getrandbits(bitlength) # Choose a really random number.
        if is_SGprime(p):
            return p    

La fonction ci-dessus cherche parmi des nombres aléatoires jusqu'à ce qu'elle trouve un nombre premier Sophie Germain. La recherche (éventuellement sans fin!) est effectuée avec une boucle `while True:` qui peut paraître étrange. L'idée est de rester dans la boucle jusqu'à ce qu'un tel nombre premier soit trouvé. Ensuite, la commande `return p` renvoie le nombre premier trouvé en sortie et arrête la boucle. Il faut faire attention avec les boucles `while True`, car elles sont structurées pour fonctionner à l'infini - s'il n'y a pas de commande de coupure de boucle telle que` return` ou `break` dans la boucle, votre ordinateur peut tourner longtemps.

In [ ]:
q = getrandSGprime(256) # A random ~256 bit Sophie Germain prime
p = 2*q + 1 # And its associated safe prime

print("p is ",p)  # Just to see what we're working with.
print("q is ",q)

Ensuite, nous trouvons une racine primitive, modulo le nombre premier sûr `p`.

In [ ]:
def findprimroot_safe(p):
    '''
    Finds a primitive root, 
    modulo a safe prime p.
    '''
    b = 2  # Start trying with 2.
    while True:  # We just keep on looking.
        if is_primroot_safe(b,p):
            return b
        b = b + 1 # Try the next base.  Shouldn't take too long to find one!

In [ ]:
g = findprimroot_safe(p)
print(g)

La paire de nombres $(g, p)$, racine primitive et nombre premier sûr, choisis par Alice ou Bob, est maintenant rendue publique. Ils peuvent publier leurs $g$ et $p$ sur un site Web public ou le crier dans les rues. Ça n'a pas d'importance. Ce ne sont que des outils pour leur algorithme de chiffrement décrit ci-dessous.

### Les nombres secrets d'Alice et Bob

Ensuite, Alice et Bob inventent chacun un nombre secret $a$ et $b$. Ils ne disent *à personne* ces nombres ; ni à l'autre ; ni à leur famille. A personne ! Ils ne les écrivent pas au tableau et ne les laissent pas sur une clé USB qu'ils perdent. Ce sont vraiment des nombres secrets.

Mais ils n'utilisent pas leurs numéros de téléphone ou de sécurité sociale. Il est préférable qu'Alice et Bob utilisent un générateur de nombre aléatoire sécurisé sur leurs ordinateurs privés distincts pour créer $a$ et $b$. Ce sont souvent des nombres de 256 bits dans la pratique, c’est ce que nous utilisons ci-dessous.

In [ ]:
a = SystemRandom().getrandbits(256)  # Alice's secret number
b = SystemRandom().getrandbits(256)  # Bob's secret number

print("Only Alice should know that a = {}".format(a))
print("Only  Bob  should know that b = {}".format(b))

print("But everyone can know p = {} and g = {}".format(p,g))

Maintenant, Alice et Bob utilisent leurs nombres secrets pour générer de nouveaux nombres.
Alice calcule le nombre
$$ A = g ^ a \text {mod } p $$
et Bob calcule le nombre
$$ B = g ^ b \text {mod } p $$

In [ ]:
A = pow(g, a, p)  # This would be computed on Alice's computer.
B = pow(g, b, p)  # This would be computed on Bob's computer.

Alice et Bob font maintenant quelque chose qui semble très étrange au première vue. Alice envoie à Bob son nouveau nombre $A$ et Bob envoie à Alice son nouveau nombre $B$. Comme ils sont éloignés les uns des autres et que la chaîne n’est pas sécurisée, on peut supposer que tout le monde connaît maintenant $A$ et $B$.

In [ ]:
print("Everyone knows \nA = {} and \nB = {}.".format(A, B))

Maintenant, Alice, sur son ordinateur privé, calcule $B^a$ mod $p$. Elle peut le faire parce que tout le monde connait $B$ et $p$, et elle connait aussi $a$.

De même, Bob, sur son ordinateur privé, calcule $A^b$ mod $p$. Il peut faire cela parce que tout le monde connait $A$ et $p$, et il connait aussi $b$.

Mais, Alice et Bob **ne partagent pas** les résultats de leurs calculs!

In [ ]:
print(pow(B, a, p))  # This is what Alice computes.

In [ ]:
print(pow(A, b, p))  # This is what Bob computes.

Woah! Qu'est-il arrivé? En termes d'exposants, c'est élémentaire ; car
$$B^a = (g^{b})^a = g^{ba} = g^{ab} = (g^a)^b = A^b$$
Donc, ces deux calculs donnent le même résultat (mod $ p $, tout au long).
Finalement, nous découvrons qu'Alice et Bob **partagent un secret**. Nous appelerons $S$ cette clé secrète.
$$S = B^a = A^b$$

In [ ]:
S = pow(B, a, p)  # Or we could have used pow(A, b, p)
print(S)

Ce nombre secret commun $S$ peut être utilisé comme clé par Alice et Bob pour communiquer.
Par exemple, ils peuvent utiliser $S$ (au besoin, convertie en chaîne) comme clé d’un chiffrement Vigenère, et discuter entre eux en sachant qu’eux seuls possèdent la clé secrète pour chiffrer et déchiffrer leurs messages.

In [ ]:
#  We use the triple-quotes for a long string, that occupies multiple lines.
#  The backslash at the end of the line tells Python to ignore the newline character.
#  Imagine that Alice has a secret message she wants to send to Bob.  
#  She writes the plaintext on her computer. 

plaintext = '''Did you hear that the American Mathematical Society has an annual textbook sale ? It's 40 percent off for members and 25 percent off for everyone else.'''
print(plaintext)

In [ ]:
# Now Alice uses the secret S (as a string) to encrypt.  
ciphertext = Vigenere_cipher(plaintext, str(S))
print(ciphertext)
# Alice sends the following ciphertext to Bob, over an insecure channel.

In [ ]:
# When Bob receives the ciphertext, he decodes it with the secret S again.
print(Vigenere_decipher(ciphertext, str(S)))

Pour faire confiance à ce protocole, vous devez être convaincu que leur secret est vraiment gardé secret! Le public a beaucoup d'informations: ils connaissent
1. le nombre premier $ p $,
2. la racine primitive $g$,
3. le nombre $ A = g^a $ (mod $ p $), et
4. le nombre $ B = g^b $ (mod $ p $).

Si le public pouvait **découvrir** soit $ a $ soit $ b $, il pourrait trouver le nombre secret (en calculant $ A^b $ ou $ B^a $ comme Alice et Bob l'ont fait).

C’est l’essence même du **problème du logarithme discret**. Si nous connaissons la valeur de $ g^a $ mod $ p $, pouvons-nous déterminer la ou les valeurs possibles de $ a $ ? S'il s'agissait d'une arithmétique ordinaire, nous dirions que $ a = \log_g (A) $. Mais c'est une arithmétique modulaire et il n'existe pas de moyen facile de comprendre de tels logarithmes. Les valeurs de $ g ^ a $ ont tendance à rebondir partout, mod $ p $, d'autant plus que nous avons choisi $ a $ assez grand (256 bits!).

La sécurité du protocole Diffie-Hellman, c'est-à-dire la sécurité du secret partagé d'Alice et de Bob, dépend de la difficulté du problème du logarithme discret. Lorsque $ p $ est un grand nombre premier sûr (par exemple 2 048 bits) et que $ a $ et $ b $ sont suffisamment grands (environ 256 bits), il semble n'y avoir aucun moyen de résoudre le problème des logarithmes discrets mod $ p $ un laps de temps raisonnable. Un jour, nous aurons peut-être des ordinateurs quantiques pour résoudre rapidement ces problèmes de logarithme discret, et le protocole Diffie-Hellman ne sera plus sécurisé. Mais pour l'instant, la clé secrète d'Alice et de Bob semble être en sécurité.

### Exercices

1. Combien y a-t-il de nombres premiers Sophie Germain entre $1$ et $10^6$? Quelle est la proportion de nombres premiers Sophie Germain dans cet intervalle ?

2. [On pense](https://en.wikipedia.org/wiki/Artin%27s_conjecture_on_primitive_roots) qu'il existe une infinité de nombres premiers $ p $ tels que $2$ soit une racine primitive mod $ p $. Etudiez la densité de tels nombres premiers. Par exemple, parmi les nombres premiers allant jusqu'à 1 000, à quelle fréquence $ 2 $ est-il une racine primitive? Cette densité semble-t-elle changer?

3. Adapter le protocole Diffie-Hellman pour travailler avec un groupe de *trois* parties souhaitant partager un secret commun. Astuce: le secret commun aura la forme $ g ^ {abc} $, et d'autres exposants tels que $ g ^ a $, $ g ^ b $, $ g ^ c $, $ g ^ {ab} $, $ g ^ {bc} $, $ g ^ {ac} $ seront des informations publiques.

4. Malheureusement, Alice et Bob ont choisi d'utiliser la racine primitive $ g = 3 $ et le nombre premier $ p = 65537 $. En écoutant leur conversation, vous interceptez ce qui suit: $ A = 40360 $ et $ B = 21002 $ et le texte chiffré est `$; 6HWD; P5LVJ99W + EH9JVx = I: V7ESpGC ^`. Si vous savez qu'ils utilisent un chiffrement Vigenère, avec une clé égale à la chaîne associée à leur secret $ S $, quel est le message en texte clair ?
Astuce: vous devriez être capable de résoudre le problème du logarithme discret avec une attaque par force brute.